In [1]:
import pandas as pd
from datetime import datetime, timedelta, date
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division
from sklearn.cluster import KMeans
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
df = pd.read_csv("data/dataset.csv",encoding = "ISO-8859-1")

Datatype Conversion

In [0]:
#Convert invoice date to datetime type
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#Creating YearMonth Field
df['InvoiceYearMonth'] = df['InvoiceDate'].map(lambda date: 100*date.year + date.month)

# **Customer Metrics**

**Customer Records**

In [0]:
#Revenue per customer
df['Revenue'] = df['UnitPrice'] * df['Quantity']

In [0]:
#Add Customer minimum purchase date
min_purchase = df.groupby(['CustomerID']).InvoiceDate.min().reset_index()
min_purchase.columns = ['CustomerID','MinPurchaseDate']
min_purchase

#Add Customer minimum purchase year month
min_purchase['MinPurchaseYearMonth'] = min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)

#Merge first purchase date and yearmonth column to our main dataframe
df = pd.merge(df, min_purchase, on='CustomerID')

In [0]:
#Categorize the user as new and existing
df['UserType'] = 'New'
df.loc[df['InvoiceYearMonth']>df['MinPurchaseYearMonth'],'UserType'] = 'Existing'

In [0]:
#New and existing user Ratio
user_ratio = df.query("UserType == 'New'").groupby(['InvoiceYearMonth'])['CustomerID'].nunique()/df.query("UserType == 'Existing'").groupby(['InvoiceYearMonth'])['CustomerID'].nunique() 
user_ratio = user_ratio.reset_index()
user_ratio = user_ratio.dropna()

In [0]:
#df_user = pd.DataFrame(df, columns = ['CustomerID', 'UserType']) 

In [0]:
#df_user.groupby(by='UserType')

**Monthly Measures and Retention Rate**

In [0]:
#Calculate Montly revenue
monthly_revenue = df.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()

#Monthly order count
monthly_sales = df.groupby(['InvoiceYearMonth'])['Quantity'].sum().reset_index()

#Average Revenue per month
monthly_order_avg = df.groupby(['InvoiceYearMonth'])['Revenue'].mean().reset_index()

#Calculating Monthly Active Customer
monthly_active = df.groupby(['InvoiceYearMonth'])['CustomerID'].nunique().reset_index()

In [0]:
print("Montly Revenue")
fig, ax = plt.subplots(figsize=(10,10))
sns.barplot(x="InvoiceYearMonth",y="Revenue",data=monthly_revenue)

In [0]:
print("Montly Sales")
fig, ax = plt.subplots(figsize=(10,10))
sns.barplot(x="InvoiceYearMonth",y="Quantity",data=monthly_sales)

In [0]:
print("Montly average revenue")
fig, ax = plt.subplots(figsize=(10,10))
sns.barplot(x="InvoiceYearMonth",y="Revenue",data=monthly_order_avg)

In [0]:
print("Montly Active Customers")
fig, ax = plt.subplots(figsize=(10,10))
sns.barplot(x="InvoiceYearMonth",y="CustomerID",data=monthly_active)

In [0]:
#calculate the Revenue per month for each user type
user_type_revenue = df.groupby(['InvoiceYearMonth','UserType'])['Revenue'].sum().reset_index()

#Out the User Type Revenue file
user_type_revenue.to_csv('User_Type_Revenue.csv')

Retention Rate

In [0]:
#identify which users are active by looking at their revenue per month
user_purchase = df.groupby(['CustomerID','InvoiceYearMonth'])['Revenue'].sum().reset_index()

#create retention matrix with crosstab
retention = pd.crosstab(user_purchase['CustomerID'], user_purchase['InvoiceYearMonth']).reset_index()

#retention

#create an array of dictionary which keeps Retained & Total User count for each month
months = retention.columns[2:]
retention_array = []
for i in range(len(months)-1):
     retention_data = {}
     selected_month = months[i+1]
     prev_month = months[i]
     retention_data['InvoiceYearMonth'] = int(selected_month)
     retention_data['TotalUserCount'] = retention[selected_month].sum()
     retention_data['RetainedUserCount'] = retention[(retention[selected_month]>0) & (retention[prev_month]>0)][selected_month].sum()
     retention_array.append(retention_data)

retention_array
    
#convert the array to dataframe and calculate Retention Rate
retention = pd.DataFrame(retention_array)
retention['RetentionRate'] = retention['RetainedUserCount']/retention['TotalUserCount']
retention.to_csv("retention_rate.csv")

# **Calculate RFM Score**

### Recency cluster using "InvoiceDate"


In [0]:
#create a generic user dataframe to keep CustomerID and new segmentation scores
recency_user = pd.DataFrame(df['CustomerID'].unique())
recency_user.columns = ['CustomerID']

#get the max purchase date for each customer and create a dataframe with it
recency_max_purchase = df.groupby('CustomerID').InvoiceDate.max().reset_index()
recency_max_purchase.columns = ['CustomerID','MaxPurchaseDate']

#we take our observation point as the max invoice date in our dataset
recency_max_purchase['Recency'] = (recency_max_purchase['MaxPurchaseDate'].max() - recency_max_purchase['MaxPurchaseDate']).dt.days

#merge this dataframe to our new user dataframe
recency_user = pd.merge(recency_user, recency_max_purchase[['CustomerID','Recency']], on='CustomerID')

#build 4 clusters for recency and add it to dataframe
kmeans = KMeans(n_clusters=4)
kmeans.fit(recency_user[['Recency']])
recency_user['RecencyCluster'] = kmeans.predict(recency_user[['Recency']])

#function for ordering cluster numbers
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

recency_user = order_cluster('RecencyCluster', 'Recency',recency_user,False)
#recency_user.head(5)


In [0]:
#sns.barplot(x='RecencyCluster',y='Recency',data=recency_user)

### Frequency cluster using "Order Count"

In [0]:
#get order counts for each user and create a dataframe with it
frequency_user = df.groupby('CustomerID').InvoiceDate.count().reset_index()
frequency_user.columns = ['CustomerID','Frequency']

#add this data to our main dataframe
recency_frequency_user = pd.merge(recency_user, frequency_user, on='CustomerID')
recency_frequency_user.head()

#k-means
kmeans = KMeans(n_clusters=4)
kmeans.fit(recency_frequency_user[['Frequency']])
recency_frequency_user['FrequencyCluster'] = kmeans.predict(recency_frequency_user[['Frequency']])

#order the frequency cluster
recency_frequency_user = order_cluster('FrequencyCluster', 'Frequency',recency_frequency_user,True)
#recency_frequency_user.head(5)

#see details of each cluster
#recency_frequency_user.groupby('FrequencyCluster')['Frequency'].describe()


### Monetory cluster using "Revenue"

In [0]:
#Create dataframe for Monetory users
monetory_user = df.groupby('CustomerID').Revenue.sum().reset_index()

#merge it with our main dataframe
recency_frequency_monetory_user = pd.merge(recency_frequency_user, monetory_user, on='CustomerID')

#apply clustering
kmeans = KMeans(n_clusters=4)
kmeans.fit(recency_frequency_monetory_user[['Revenue']])
recency_frequency_monetory_user['RevenueCluster'] = kmeans.predict(recency_frequency_monetory_user[['Revenue']])


#order the cluster numbers
recency_frequency_monetory_user = order_cluster('RevenueCluster', 'Revenue',recency_frequency_monetory_user,True)

#recency_frequency_monetory_user.head()

### Overall Score by Summing all the cluster groups

In [21]:
#calculate overall score and use mean() to see details
recency_frequency_monetory_user['OverallScore'] = recency_frequency_monetory_user['RecencyCluster'] + recency_frequency_monetory_user['FrequencyCluster'] + recency_frequency_monetory_user['RevenueCluster']
recency_frequency_monetory_user.groupby('OverallScore')['Recency','Frequency','Revenue'].mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Recency,Frequency,Revenue
OverallScore,,,
0,308.960239,21.980119,328.607117
1,190.586710,33.243112,550.290147
2,81.665339,46.433267,909.752841
3,21.172276,69.945807,1236.218728
4,13.509091,302.438961,4380.503506
5,7.400000,491.435294,13810.254706
6,7.307692,985.230769,38669.643077
7,2.555556,2432.555556,98056.746667
8,1.333333,4372.000000,156394.183333


# **Customer Segmentation**

In [0]:
recency_frequency_monetory_user['Segment'] = 'Low-Value'
recency_frequency_monetory_user.loc[recency_frequency_monetory_user['OverallScore']>2,'Segment'] = 'Mid-Value' 
recency_frequency_monetory_user.loc[recency_frequency_monetory_user['OverallScore']>4,'Segment'] = 'High-Value'

In [23]:
recency_frequency_monetory_user['Revenue'].max()

279489.0199999999

In [25]:
#Revenue vs Frequency
rvf_graph = recency_frequency_monetory_user.query("Revenue > 50000 and Frequency > 1000")
rvf_graph['Revenue'].max()

279489.0199999999

In [50]:
#Use this for Visualization : Customer Segmentation Information
df_customer_segmentation = pd.DataFrame(recency_frequency_monetory_user, columns = ['CustomerID','RecencyCluster','FrequencyCluster','RevenueCluster','OverallScore','Segment'])
df_customer_segmentation.to_csv("Customer_Segmentation.csv")
df_customer_segmentation

,CustomerID,RecencyCluster,FrequencyCluster,RevenueCluster,OverallScore,Segment
0,17850.0,0,1,0,1,Low-Value
1,15808.0,0,1,0,1,Low-Value
2,13047.0,3,1,0,4,Mid-Value
3,12583.0,3,1,0,4,Mid-Value
4,14688.0,3,1,0,4,Mid-Value
...,...,...,...,...,...,...
4367,12748.0,3,3,1,7,High-Value
4368,17841.0,3,3,1,7,High-Value
4369,17450.0,3,1,3,7,High-Value
4370,18102.0,3,1,3,7,High-Value


# **Customer Lifetime Value**

In [0]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).apply(lambda x: x.date())

3 months and 6 months dataframe

In [0]:
df_3m = df[(df.InvoiceDate < date(2011,6,1)) & (df.InvoiceDate >= date(2011,3,1))].reset_index(drop=True)
df_6m = df[(df.InvoiceDate >= date(2011,6,1)) & (df.InvoiceDate < date(2011,12,1))].reset_index(drop=True)

In [0]:
#create df_user for assigning clustering
df_user = pd.DataFrame(df_3m['CustomerID'].unique())
df_user.columns = ['CustomerID']

#order cluster method
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

RFM Score for 3 months

In [0]:
#Recency dataframe
df_max_purchase = df_3m.groupby('CustomerID').InvoiceDate.max().reset_index()
df_max_purchase.columns = ['CustomerID','MaxPurchaseDate']
df_max_purchase['Recency'] = (df_max_purchase['MaxPurchaseDate'].max() - df_max_purchase['MaxPurchaseDate']).dt.days
df_user = pd.merge(df_user, df_max_purchase[['CustomerID','Recency']], on='CustomerID')

In [0]:
#Recency Cluster
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_user[['Recency']])
df_user['RecencyCluster'] = kmeans.predict(df_user[['Recency']])

df_user = order_cluster('RecencyCluster', 'Recency',df_user,False)

In [0]:
#Frequency dataframe
df_frequency = df_3m.groupby('CustomerID').InvoiceDate.count().reset_index()
df_frequency.columns = ['CustomerID','Frequency']
df_user = pd.merge(df_user, df_frequency, on='CustomerID')

In [0]:
#Frequency cluster
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_user[['Frequency']])
df_user['FrequencyCluster'] = kmeans.predict(df_user[['Frequency']])

df_user = order_cluster('FrequencyCluster', 'Frequency',df_user,True)

In [0]:
#Revenue dataframe
df_3m['Revenue'] = df_3m['UnitPrice'] * df_3m['Quantity']
df_revenue = df_3m.groupby('CustomerID').Revenue.sum().reset_index()
df_user = pd.merge(df_user, df_revenue, on='CustomerID')

In [0]:
#Revenue dataframe
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_user[['Revenue']])
df_user['RevenueCluster'] = kmeans.predict(df_user[['Revenue']])
df_user = order_cluster('RevenueCluster', 'Revenue',df_user,True)
#df_user

In [0]:
#overall scoring
df_user['OverallScore'] = df_user['RecencyCluster'] + df_user['FrequencyCluster'] + df_user['RevenueCluster']
df_user['Segment'] = 'Low-Value'
df_user.loc[df_user['OverallScore']>2,'Segment'] = 'Mid-Value' 
df_user.loc[df_user['OverallScore']>4,'Segment'] = 'High-Value' 
#df_user

In [0]:
#Revenue for 6 months
df_6m['Revenue'] = df_6m['UnitPrice'] * df_6m['Quantity']
df_user_6m = df_6m.groupby('CustomerID')['Revenue'].sum().reset_index()
df_user_6m.columns = ['CustomerID','m6_Revenue']
#df_user_6m

In [0]:
df_merge = pd.merge(df_user, df_user_6m, on='CustomerID', how='left')
df_merge = df_merge.fillna(0)

df_graph = df_merge.query("m6_Revenue < 30000")
#df_graph

In [0]:
#remove outliers
df_merge = df_merge[df_merge['m6_Revenue']<df_merge['m6_Revenue'].quantile(0.99)]


#creating 3 clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(df_merge[['m6_Revenue']])
df_merge['LTVCluster'] = kmeans.predict(df_merge[['m6_Revenue']])

#order cluster number based on LTV
df_merge = order_cluster('LTVCluster', 'm6_Revenue',df_merge,True)

#creatinga new cluster dataframe
df_cluster = df_merge.copy()
df_cluster.head()
#Low value segment form cluster 1,mid value cluster 2 and high value cluster 3

#see details of the clusters
df_cluster.groupby('LTVCluster')['m6_Revenue'].describe()


In [0]:
#convert categorical columns to numerical
df_class = pd.get_dummies(df_cluster)


#calculate and show correlations
corr_matrix = df_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

# #create X and y, X will be feature set and y is the label - LTV
x = df_class.drop(['LTVCluster','m6_Revenue'],axis=1)
#X is passing all the feature values
y = df_class['LTVCluster']
#y has the LTV Cluster value
#x = 1965 rows

#split training and test sets with 80 training and 20 testing
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=70)

#x
#y
#X_train = Contains all the columns 
#X_test = Contains all the columns 
#y_train = LTV cluster column only
#y_test = LTV Cluster Column only

In [43]:
#XGBoost Multiclassification Model
ltv_xgb_model = xgb.XGBClassifier(max_depth=4, learning_rate=0.2,objective= 'multi:softprob',n_jobs=-1).fit(X_train, y_train)
#after max depth of 6 training accuracy is 1 constant and test varies little like 0.86 and 0.87

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(ltv_xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(ltv_xgb_model.score(X_test[X_train.columns], y_test)))

y_pred = ltv_xgb_model.predict(X_test)
#print(classification_report(y_test, y_pred))
y_test,y_pred

Accuracy of XGB classifier on training set: 0.98
Accuracy of XGB classifier on test set: 0.86


(932     0
 1511    0
 1286    0
 280     0
 1655    0
        ..
 1949    1
 1544    0
 830     0
 1935    1
 135     0
 Name: LTVCluster, Length: 402, dtype: int64,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0,

In [44]:
X_test['y_test'] = y_test
X_test['y_pred'] = y_pred
X_test.to_csv('CLV.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
clv = pd.read_csv("CLV.csv")

In [0]:
# Add column 'customer_lifetime_value' 
conditions = [
              clv['y_pred'] == 0,
              clv['y_pred'] == 1,
              clv['y_pred'] == 2
]

choices = ['LowCLV','MedCLV','HighCLV']

clv['Customer_Lifetime_value'] = np.select(conditions, choices)

In [0]:
clv.to_csv('CLV.csv')